# ДЗ2. Мультимодальный адаптер к Qwen3-0.6B

**Описание задания**

В этом задании вы подключите внешнюю модальность (аудио или изображение) к языковой модели Qwen3-0.6B через небольшой обучаемый адаптер. Веса Qwen и выбранного предобученного энкодера мы замораживаем, обучается только адаптер.

- Трек B (изображение → текст)

Описываем изображения (image captioning датасет) с помощью Qwen. Энкодер: любая vision-модель.

**Задачи:**

1.   Заморозить параметры `QWEN` и предобученного энкодера (аудио или vision).

2.   Создать и обучить адаптер, который сжимает временную / пространственную размерность признаков и проецирует их в скрытое пространство `Qwen`.

3.   Подготовить данные для обучения и валидации модели.

4.   Реализовать и сравнить несколько стратегий пулинга в адаптере (например, сжатие временной размерности для аудио или пространственной - для изображений).

5.   Использовать `BERTScore` для оценки качества модели на отложенном датасете.

**Основные этапы задания**

*   Подготовка данных: для "изображение → текст" загрузите датасет для image captioning (например, `Flickr8k`), обработайте данные и создайте DataLoader для батчевого обучения. Рекомендуется выполнить полную предобработку данных (прекомпьют), чтобы сократить время обработки на этапе обучения.
*   Реализация адаптера: создайте класс, который сжимает последовательность векторов.
*   Интеграция с `QWEN`: реализуйте обработку входов и передачу через `QWEN`.
*   Обучение модели: настройте процесс обучения с использованием `Cross Entropy Loss` и teacher forcing.
*   Оценка: вычислите BERTScore между сгенерированными и реальными текстами на валидационном датасете.

> **Внимание!** Последующие ячейки с условиями оформлены (название классов и переменных, инструкциии и комментарии) в ключе работы по треку А (аудио → текст). Если вы выбираете работать с vision задачей, можете ориентироваться содержательно на представленные кодовые сниппеты, но видоизменять их под свою задачу.

### Сеттинг

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [3]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig
)
from datasets import load_dataset
import random
import numpy as np
import pandas as pd
from bert_score import BERTScorer
from tqdm import tqdm

/home/cloexec/Desktop/HSE/multimodal/HW1/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


**Загрузка данных**

Если вы выбираете трек с аудио, используйте датасет `AudioCaps`.

Я буду использовать Flickr8k.

In [5]:
# your code here
# use flickr8k dataset

ds = load_dataset("jxie/flickr8k").with_format("torch")

In [6]:
ds['train'][0]

{'image': tensor([[[ 38,  64,  78,  ...,  52,  60,  41],
          [ 59,  91,  54,  ...,  37,  57,  60],
          [ 90,  79,  82,  ...,  43,  36,  18],
          ...,
          [223, 218, 211,  ..., 215, 215, 210],
          [241, 234, 233,  ..., 231, 218, 205],
          [237, 244, 241,  ..., 226, 226, 223]],
 
         [[ 31,  50,  73,  ...,  48,  56,  37],
          [ 37,  59,  34,  ...,  36,  52,  50],
          [ 66,  50,  67,  ...,  28,  29,  18],
          ...,
          [223, 220, 213,  ..., 217, 217, 212],
          [243, 236, 238,  ..., 241, 228, 214],
          [239, 248, 247,  ..., 230, 229, 226]],
 
         [[ 25,  49,  67,  ...,  37,  47,  28],
          [ 49,  60,  25,  ...,  32,  49,  49],
          [ 56,  46,  60,  ...,  21,  21,   8],
          ...,
          [221, 217, 210,  ..., 229, 229, 225],
          [240, 233, 234,  ..., 242, 230, 221],
          [238, 247, 245,  ..., 239, 238, 233]]], dtype=torch.uint8),
 'caption_0': 'A black dog is running after a white do

### Задание 1. Создание и обучение AudioConvAdapter (3 балла)

1. Загрузите модель **Qwen3-0.6B** и заморозьте её параметры.  
2. Загрузите предобученный аудио-энкодер (параметры также должны быть заморожены). Вы можете выбрать одну из следующих моделей: **HuBERT**, **wav2vec2**, или **Whisper Encoder**. Учтите особенности выбранной модели:  
   - Например, `wav2vec2-large-960h-lv60-self` использует `flash_attention`, которая работает только с GPU архитектуры Ampere и с типом данных float16. Это может вызвать сложности, если используется другое оборудование.  
3. Реализуйте класс `AudioConvAdapter`, который уменьшает размерность последовательности аудио по времени и переводит её в текстовое пространство модели QWEN.

In [7]:
# Load
qwen_model_name = "Qwen/Qwen3-0.6B"

qwen_tokenizer =  AutoTokenizer.from_pretrained(qwen_model_name, use_fast=True)
qwen_model = AutoModelForCausalLM.from_pretrained(qwen_model_name).to(device)

In [8]:
# Test
txt = "Hello world!"
tok = qwen_tokenizer(txt, return_tensors='pt')
tok['input_ids'] = tok['input_ids'].to(device)
tok['attention_mask'] = tok['attention_mask'].to(device)
out = qwen_model.generate(**tok)
qwen_tokenizer.decode(out[0])

"Hello world! This is a simple example of the web application. It's a good idea to have a simple web"

In [9]:
# Freeze
qwen_model.eval()
for p in qwen_model.parameters():
    p.requires_grad = False

In [10]:
from transformers import AutoImageProcessor, CLIPVisionModel

# Load
vision_model_name = "openai/clip-vit-base-patch32"
image_processor = AutoImageProcessor.from_pretrained(vision_model_name, use_fast=True)
vision_encoder = CLIPVisionModel.from_pretrained(vision_model_name).to(device)

In [11]:
# Freeze
vision_encoder.eval()
for p in vision_encoder.parameters():
    p.requires_grad = False

In [12]:
from src.adapter import VisionAdapter

In [13]:
qwen_hidden = qwen_model.config.hidden_size
vision_hidden = vision_encoder.config.hidden_size

vision_hidden, qwen_hidden

(768, 1024)

In [14]:
adapter_hidden_dim = 1024  # внутренняя размерность adapter’а
vision_adapter = VisionAdapter(vision_hidden, qwen_hidden)# your code here

print("Trainable parameters in adapter: {:,}".format(sum(p.numel() for p in vision_adapter.parameters() if p.requires_grad)))

Trainable parameters in adapter: 2,561,280


### Задание 2. Подготовка данных (2 балла)

In [15]:
from src.dataset import Flickr8kCaptionDataset, collate_fn_vision

In [16]:
train_dataset = Flickr8kCaptionDataset(ds["train"])
val_dataset   = Flickr8kCaptionDataset(ds["test"] if "test" in ds else ds["validation"])

# DataLoaders (note: collate_fn needs processor+tokenizer, so use lambda/partial)
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    # num_workers=4,
    # pin_memory=True,
    collate_fn=lambda b: collate_fn_vision(b, image_processor, qwen_tokenizer, device),
)
val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    # num_workers=4,
    # pin_memory=True,
    collate_fn=lambda b: collate_fn_vision(b, image_processor, qwen_tokenizer, device),
)


# Проверка
batch = next(iter(train_loader))
print(batch["pixel_values"].shape)      # (B, 3, H, W)
print(batch["input_ids"].shape)         # (B, L)
print(batch["attention_mask"].shape)    # (B, L)
print('Captions 1-4:', end='\n\t')
print(*batch["captions"][:4], sep='\n\t')

torch.Size([4, 3, 224, 224])
torch.Size([4, 21])
torch.Size([4, 21])
Captions 1-4:
	A man grips the underhang of a rock .
	A man in a wheelchair riding to the park .
	A big brown dog runs with a stick in his mouth , and a big black down runs behind him .
	A brown dogs licks its black nose .


In [17]:
vision_adapter = vision_adapter.to(device).train()
vision_encoder.eval()
qwen_model.eval()

with torch.no_grad():
    vis_out = vision_encoder(pixel_values=batch["pixel_values"])
    vis_tokens = vis_out.last_hidden_state  # (B, S, Dv)

visual_embeds = vision_adapter(vis_tokens)  # (B, T, Dq)
print("visual_embeds:", visual_embeds.shape)


visual_embeds: torch.Size([4, 16, 1024])


## Задание 3. Трейн QwenAudioDescription (3 балла)


1. Напишите класс `QwenAudioDescriptionTrainer`, который будет включать в себя:
   - Обучение адаптера (`train_one_epoch`).
   - Валидацию (`validate`).
   - Генерацию описания аудио (`generate`).
2. Реализуйте процесс обучения, который объединяет аудио-эмбеддинги и текстовые токены, а затем передаёт их в QWEN для предсказания текстов.
3. Используйте Cross Entropy Loss для оптимизации аудио-адаптера. Остальные параметры модели остаются замороженными.


In [18]:
# # Создайте ID для специального токена [AUDIO]
# audio_token_id = qwen_tokenizer("[AUDIO]", add_special_tokens=False)["input_ids"][0]
# special_token_id = audio_token_id

# class QwenAudioDescriptionTrainer:
#     def __init__(self, qwen_model, qwen_tokenizer, audio_encoder, audio_adapter, lr=1e-4):
#         self.qwen_model = qwen_model
#         self.qwen_tokenizer = qwen_tokenizer
#         self.audio_encoder = audio_encoder
#         self.audio_adapter = audio_adapter

#         # Создайте оптимизатор Adam (только для audio_adapter)
#         # self.optimizer = ...

#     def train_one_epoch(self, train_loader):
#         self.audio_adapter.train()
#         total_loss = 0.0

#         for batch in tqdm(train_loader, desc="Training"):
#             audio_inputs, texts = batch  # (audio_inputs, список строк)

#             # Подготовьте input_values и attn_mask - маску для аттеншена (только аудио, тк текст предсказываем)

#             # *Этот шаг выполняется, если аудио векторы не вычислялись при формировании батча.
#             # *Прогните через audio_encoder (заморожен) без градиентов
#             with torch.no_grad():
#                 # *audio_hidden_states = ...
#                 pass

#             # Пропустите скрытые состояния через аудио-адаптер
#             # Токенизируйте текстовые данные и примите QWEN эмбеддер
#             # Соберите all_embeddings для модели cat(audio, text)
#             # Таргетом являются лейблы текстовых токенов (описаний аудио)
#             # Пропустите данные через модель QWEN и вычислите лосс
#             # Выполните шаг оптимизации

#             total_loss += loss.item()

#         avg_loss = total_loss / len(train_loader)
#         return avg_loss

#     def validate(self, val_loader):
#         self.audio_adapter.eval()
#         val_loss = 0.0

#         with torch.no_grad():
#             for batch in tqdm(val_loader, desc="Validating"):
#                 audio_inputs, texts = batch

#                 # Аналогично train, но без backward
#                 # your code here

#                 pass

#         return val_loss / len(val_loader)

#     def generate(self, audio_inputs):
#         """
#         Генерация описания для одного аудио.
#         """
#         self.audio_adapter.eval()
#         with torch.no_grad():

#             # 1) input_values + attn_mask
#             # 2) audio_encoder -> audio_adapter
#             # 3) Склейте audio_embeds + небольшой pseudo_input_ids
#             # 4) Сгенерируйте текст
#             # generated_ids = ...
#             # generated_text = ...

#         return generated_text


In [19]:
from src.train import QwenVisionCaptionTrainer

In [20]:
trainer = QwenVisionCaptionTrainer(
    qwen_model=qwen_model,
    qwen_tokenizer=qwen_tokenizer,
    vision_encoder=vision_encoder,
    vision_adapter=vision_adapter.to(device),
    device=device,
    lr=1e-4,
)

In [21]:
epochs = 2
for ep in range(1, epochs + 1):
    tr_loss = trainer.train_one_epoch(train_loader)
    va_loss = trainer.validate(val_loader)
    print(f"epoch={ep} train_loss={tr_loss:.4f} val_loss={va_loss:.4f}")

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:17<00:00, 14.02it/s]


epoch=1 train_loss=3.3811 val_loss=3.0703


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:17<00:00, 14.21it/s]

epoch=2 train_loss=3.0188 val_loss=2.9380


> Убедиться, что:
>
> При обучении лосс падает.
>
> При валидации всё аналогично, только без backward.
>
>  При генерации появляется текст (возможно, не самый качественный - это зависит от данных и количества эпох).

In [23]:
epochs = 10
for ep in range(1, epochs + 1):
    tr_loss = trainer.train_one_epoch(train_loader)
    va_loss = trainer.validate(val_loader)
    print(f"epoch={ep} train_loss={tr_loss:.4f} val_loss={va_loss:.4f}")

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:17<00:00, 14.27it/s]


epoch=1 train_loss=2.8919 val_loss=2.8710


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:17<00:00, 14.18it/s]


epoch=2 train_loss=2.7888 val_loss=2.8280


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:17<00:00, 14.09it/s]


...

epoch=8 train_loss=2.4053 val_loss=2.7602


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:17<00:00, 14.38it/s]


epoch=9 train_loss=2.3043 val_loss=2.7319


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:17<00:00, 13.99it/s]

epoch=10 train_loss=2.1957 val_loss=2.7852


## Задание 4. Валидация (2 балла)

1. Используйте валидационный набор данных, чтобы проверить, насколько хорошо модель генерирует текстовые описания для аудио/картинок.

2. Реализуйте процесс генерации текстов для всех аудио/картинок из валидационного набора.

3. Используйте метрику **BERTScore** для оценки качества сгенерированных описаний.

4. Отобразите примеры сгенерированных текстов и сравните их с истинными описаниями (references).

In [25]:
preds, refs = [], []
for batch in tqdm(val_loader, desc="Generating"):
    preds.extend(trainer.generate(batch["pixel_values"], max_new_tokens=40, num_beams=3))
    refs.extend(batch["captions"])

Generating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [05:14<00:00,  1.26s/it]


In [26]:
from bert_score import BERTScorer

scorer = BERTScorer(model_type="roberta-large", lang="en", rescale_with_baseline=True)  # rescale supported by bert-score
P, R, F1 = scorer.score(preds, refs)

print(f"BERTScore P={P.mean().item():.4f} R={R.mean().item():.4f} F1={F1.mean().item():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore P=-0.3885 R=0.3047 F1=-0.0696


In [27]:
for i in range(5):
    print("\nREF:", refs[i])
    print("HYP:", preds[i])


REF: The dogs are in the snow in front of a fence .
HYP: A brown dog is running in the snow . The other dog is a black dog . The dogs are both dogs . The dogs are both dogs . The dogs are both dogs . The dogs are both dogs

REF: a brown and white dog swimming towards some in the pool
HYP: a black and white dog jumping into a pool . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

REF: A man and a woman in festive costumes dancing .
HYP: A man in a white shirt and a black shirt is holding a sign that says “I’m a man.” ” . . . . . . . . . . . . . . . . .

REF: A couple of people sit outdoors at a table with an umbrella and talk .
HYP: A man and a woman sit on a bench . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

REF: A man is wearing a Sooners red football shirt and helmet .
HYP: A black and white football player in a red jersey . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
